In [1]:
import os

curr_folder = os.getcwd() + "/"
datasets_folder = curr_folder[:-6] + "Datasets/"
stats_results_folder = curr_folder + "Results/"
print(datasets_folder)
print(stats_results_folder)

/home/ipsilva/Repositories/dynamic-projections/Datasets/
/home/ipsilva/Repositories/dynamic-projections/Stats/Results/


In [2]:
datasets_subfolders = []
for i in os.walk(datasets_folder):
    datasets_subfolders.append(i[0])

datasets_subfolders = datasets_subfolders[1:]
datasets_subfolders

['/home/ipsilva/Repositories/dynamic-projections/Datasets/gaussians',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/quickdraw',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/sorts',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/walk',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/nnset',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/fashion',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/cifar10cnn',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/esc50',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/qtables',
 '/home/ipsilva/Repositories/dynamic-projections/Datasets/cartolastd']

In [3]:
dataset_number = 7
selected_dataset = datasets_subfolders[dataset_number]
selected_dataset

'/home/ipsilva/Repositories/dynamic-projections/Datasets/esc50'

In [4]:
from os import listdir
from os.path import isfile, join

files = [f for f in listdir(selected_dataset) if isfile(join(selected_dataset, f))]
files[:5]

['esc50-100.csv',
 'esc50-88.csv',
 'esc50-68.csv',
 'esc50-90.csv',
 'esc50-102.csv']

In [5]:
from sklearn.preprocessing import MinMaxScaler
from itertools import product
import pandas as pd
import numpy as np

df = pd.DataFrame()

obs_per_timestep = []
for file in files:
    file_loc = selected_dataset + "/" + file
    ndf = pd.read_csv(file_loc)
    df = df.append(ndf)
    obs_per_timestep.append(ndf.shape[0])
    
median_obs_per_timestep = np.median(obs_per_timestep)
dev_obs_per_timestep = np.std(obs_per_timestep)

min_max_scaler = MinMaxScaler()
df = pd.DataFrame(min_max_scaler.fit_transform(df.drop(columns=df.columns[0], axis=1)))
print(df.shape, df.head())

(34560, 128)         0         1         2         3         4         5         6    \
0  0.893591  0.881061  0.859159  0.834986  0.814445  0.784877  0.758748   
1  0.878150  0.870891  0.847291  0.813527  0.778361  0.735661  0.702460   
2  0.516475  0.529808  0.540790  0.551289  0.562773  0.565186  0.566901   
3  0.395895  0.406114  0.416875  0.429916  0.445495  0.454088  0.461425   
4  0.501981  0.521588  0.540918  0.559567  0.575336  0.575612  0.570660   

        7         8         9    ...       118       119       120       121  \
0  0.739336  0.728934  0.727744  ...  0.595333  0.596239  0.597528  0.598795   
1  0.683019  0.677538  0.683377  ...  0.537299  0.533330  0.525200  0.515340   
2  0.569697  0.573325  0.576822  ...  0.707336  0.709125  0.711539  0.714764   
3  0.469233  0.477615  0.484979  ...  0.672224  0.670170  0.664536  0.656302   
4  0.566212  0.564015  0.562138  ...  0.743481  0.748923  0.753665  0.756441   

        122       123       124       125       126    

In [6]:
from sklearn.decomposition import PCA

def intrisic_dim(dataframe, target_variance):
    pca = PCA()

    principalComponents = pca.fit_transform(df)

    list_variance = pca.explained_variance_ratio_

    final_var = 0
    dims = 0
    for var in list_variance:
        final_var += var
        dims += 1
        if final_var > target_variance:
            break
    return dims, final_var, principalComponents[:dims]

In [7]:
target_variance = 0.9

(dims, final_var, components) = intrisic_dim(df, target_variance)

print("Total time-steps:", len(obs_per_timestep))
print("Registers per time-step (median):", median_obs_per_timestep)
print("Registers per time-step (deviation):", dev_obs_per_timestep)
print("Total dimensions:", df.shape[1])
print("Intrinsic dimensions:", dims)
print("Total variance:", final_var)

Total time-steps: 108
Registers per time-step (median): 320.0
Registers per time-step (deviation): 0.0
Total dimensions: 128
Intrinsic dimensions: 3
Total variance: 0.9229026005300335


In [8]:
import matplotlib.pyplot as plt

# identifying distributions
hists_ax = df.hist(bins=20, grid=False, figsize=(10,10), density=True)

for arr in hists_ax:
    for ax in arr:
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        
fig_name = selected_dataset.split('/')[-1]
plt.suptitle(fig_name + " probability distributions", fontsize=15, weight='roman')
plt.tight_layout()
plt.subplots_adjust(top=0.925)
plt.savefig(stats_results_folder+"attribute_histograms/"+fig_name+"_prob_dist")